In [1]:
import pandas as pd
import os, sys, pandas, pathlib, time
from collections import defaultdict
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torchvision.transforms import v2
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import cv2
import numpy as np
from tqdm import tqdm, trange
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

Image.MAX_IMAGE_PIXELS = None #Throws error if size of images exceeds this number

In [3]:
#Creates dictionary mapping labels(string) to vector(one-hot encoding)
all_labels = list(np.load('materials/UBC-OCEAN_CS640/all_labels.npy'))
num_classes = len(all_labels)
label_dict = defaultdict(lambda: torch.zeros(num_classes))
for i, label in enumerate(all_labels):
    label_dict[label][i] = 1

In [4]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, num_classes, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            num_classes (int): Total number of classes.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.labels_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.num_classes = num_classes
        self.transform = transform

    def __len__(self):
        return len(self.labels_frame.index)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, str(self.labels_frame.iloc[idx, 0])) + '.jpg'
        image = Image.open(img_name)
        #image = Image.open(img_name)
        label = self.labels_frame.iloc[idx, 1]

        # Convert label to one-hot encoding
        one_hot = label_dict[label]

        if self.transform:
            image = self.transform(image)

        return image, one_hot

In [5]:
    # change the size only if necessary
transform = v2.Compose([v2.Resize(256, antialias = True),
                            v2.CenterCrop(224),
                            v2.ToImage(),
                            v2.ToDtype(torch.float32, scale = True),
                            v2.Normalize(mean = [0.4887, 0.4266, 0.4855], std = [0.4212, 0.3790, 0.4169])])

In [6]:
batch_size = 8

# Create the dataset
train_dataset = CustomDataset(csv_file='materials/UBC-OCEAN_CS640/train.csv', root_dir='materials/UBC-OCEAN_CS640/train_images_compressed_80/', num_classes=num_classes, transform=transform)
test_dataset = CustomDataset(csv_file='materials/UBC-OCEAN_CS640/test.csv', root_dir='materials/UBC-OCEAN_CS640/test_images_compressed_80/', num_classes=num_classes, transform=transform)

# Create a DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

# Now you can use this dataloader in your training loop

In [7]:
#Size of the reshaped first image in the data set
#(RGB * width * height)
print(f'Numer of training examples: {train_dataset.__len__()}')
print(f'Numer of testing examples: {test_dataset.__len__()}')
print(f'Type of pairing: {type(train_dataset.__getitem__(0))}')
print(f'Number of inputs + outputs: {len(train_dataset.__getitem__(0))}')
print(f'Size of input: {train_dataset.__getitem__(0)[0].size()}')
print(f'First input(normalized): {train_dataset.__getitem__(0)[0]}')
print(f'First output(one-hot encoding): {train_dataset.__getitem__(0)[1]}')

Numer of training examples: 430
Numer of testing examples: 108
Type of pairing: <class 'tuple'>
Number of inputs + outputs: 2
Size of input: torch.Size([3, 224, 224])
First input(normalized): Image([[[-1.1603, -1.1603, -1.1603,  ...,  0.7205,  0.7298,  0.7391],
        [-1.1603, -1.1603, -1.1603,  ...,  0.7298,  0.7298,  0.7391],
        [-1.1603, -1.1603, -1.1603,  ...,  0.7391,  0.7298,  0.7298],
        ...,
        [-1.1603, -1.1603, -1.1603,  ...,  0.7018,  0.6925,  0.7577],
        [-1.1603, -1.1603, -1.1603,  ...,  0.7391,  0.8508,  0.9346],
        [-1.1603, -1.1603, -1.1603,  ...,  0.8694,  0.9532,  0.9625]],

       [[-1.1256, -1.1256, -1.1256,  ...,  0.4575,  0.4886,  0.5196],
        [-1.1256, -1.1256, -1.1256,  ...,  0.4782,  0.4989,  0.5093],
        [-1.1256, -1.1256, -1.1256,  ...,  0.4886,  0.5093,  0.4989],
        ...,
        [-1.1256, -1.1256, -1.1256,  ...,  0.4782,  0.4886,  0.6645],
        [-1.1256, -1.1256, -1.1256,  ...,  0.5920,  0.8818,  1.0783],
        [-

In [8]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU is available


In [9]:
class CancerCNN_1(nn.Module):
    def __init__(self):
        super(CancerCNN_1, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        # Adjust the size of the fully connected layer
        self.fc1 = nn.Linear(128 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, 5) # Adjust according to the number of classes
        self.pool = nn.MaxPool2d(2, 2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(-1, 128 * 28 * 28) # Adjust this flattening based on the output size
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [10]:
# Step 4: Initialize the CNN Model
model_1 = CancerCNN_1().float()
print(model_1)

model_1 = model_1.to(device)

# Step 5: Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_1.parameters(), lr=0.0001)

CancerCNN_1(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=100352, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=5, bias=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu): ReLU()
)


In [11]:
num_epochs = 10

losses = []


# Train the Model
#try:
#    model_1 = torch.load('Cancer_CNN_1.pt')
#    img = mpimg.imread('loss_1.png')
#    imgplot = plt.imshow(img)
#    plt.show()

#except:
model_1.train()
for epoch in tqdm(range(num_epochs)):  # Number of epochs
    for data, target in tqdm(train_dataloader):
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = model_1(data)
        print(f'output: {output}')
        print(f'target: {target.type_as(output)}\n')
        loss = criterion(output, target.type_as(output)) # Ensuring target is same type as output
        loss.backward()
        optimizer.step()
    losses.append(loss.item())
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')
torch.save(model_1, 'Cancer_CNN_1.pt')
    
fig = plt.figure()
ax = plt.subplot(111)
ax.plot(losses)
plt.title('Loss')
plt.show()
fig.savefig('loss_1.png')

  0%|          | 0/10 [00:00<?, ?it/s]


output: tensor([[-0.0112,  0.0270,  0.0812, -0.0124, -0.0536],
        [-0.0200,  0.0084,  0.0711, -0.0239, -0.0625],
        [-0.0216,  0.0171,  0.0723, -0.0152, -0.0481],
        [-0.0128,  0.0155,  0.0561, -0.0197, -0.0493],
        [-0.0215,  0.0174,  0.0724, -0.0153, -0.0486],
        [-0.0070,  0.0106,  0.0569, -0.0281, -0.0617],
        [ 0.0022,  0.0163,  0.0691, -0.0241, -0.0567],
        [-0.0111,  0.0083,  0.0544, -0.0170, -0.0682]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
target: tensor([[1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0.]], device='cuda:0')



  2%|▏         | 1/54 [00:19<17:28, 19.78s/it]

output: tensor([[ 0.8792,  0.1461, -0.8902,  0.0547, -0.1971],
        [ 1.0192,  0.3669, -1.1197,  0.1486, -0.4566],
        [ 0.9711,  0.3886, -1.1271,  0.1240, -0.4296],
        [ 0.9414,  0.4748, -1.1012,  0.1872, -0.5804],
        [ 0.9651,  0.4685, -1.1164,  0.1849, -0.5736],
        [ 0.9910,  0.4617, -1.1308,  0.1746, -0.5774],
        [ 0.8197,  0.1631, -0.8632,  0.0524, -0.1771],
        [ 0.9808,  0.4615, -1.1213,  0.1788, -0.5766]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
target: tensor([[1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0.]], device='cuda:0')



output: tensor([[ 0.4539,  0.6682, -1.7147,  0.4490,  0.2355],
        [ 0.4455,  0.8523, -1.7251,  0.4560,  0.1979],
        [ 0.6650,  0.3845, -1.6138,  0.4167,  0.2545],
        [ 0.4661,  0.8724, -1.7468,  0.4329,  0.1952],
        [ 0.5339,  0.7703, -1.7344,  0.4346,  0.2340],
        [ 0.7627,  0.1897, -1.3472,  0.3284,  0.2271],
        [ 0.4467,  0.8564, -1.7408,  0.4580,  0.2012],
        [ 0.6743,  0.3481, -1.4614,  0.3467,  0.2102]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
target: tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.]], device='cuda:0')



...

output: tensor([[ 1.5600, -0.4134, -0.3055,  0.1376, -0.5334],
        [ 1.5677, -0.3585, -0.3753,  0.0826, -0.4961],
        [ 1.3643, -0.3567, -0.2825,  0.1336, -0.4743],
        [ 1.1696, -0.3566, -0.3724,  0.2133, -0.1444],
        [ 1.3548, -0.4582, -0.2725,  0.1547, -0.3331],
        [ 1.5056, -0.4410, -0.3497,  0.1795, -0.3910],
        [ 0.9543, -0.3039, -0.3132,  0.2358,  0.0121],
        [ 1.3519, -0.3482, -0.2736,  0.1253, -0.4561]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
target: tensor([[0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0.]], device='cuda:0')



output: tensor([[ 0.6359, -0.2813, -0.0850,  0.2711, -0.0415],
        [ 0.6919, -0.3327, -0.0798,  0.3040, -0.1446],
        [ 0.6358, -0.2813, -0.0850,  0.2711, -0.0414],
        [ 0.6358, -0.2813, -0.0850,  0.2711, -0.0414],
        [ 1.2090, -0.5114,  0.0220,  0.1754, -0.4896],
        [ 1.0810, -0.5208, -0.0474,  0.1843, -0.2033],
        [ 0.6488, -0.2874, -0.0944,  0.2792, -0.0366],
        [ 0.6663, -0.3188, -0.0768,  0.3009, -0.0787]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
target: tensor([[0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.]], device='cuda:0')



output: tensor([[ 0.6973, -0.4057,  0.0027,  0.3547, -0.2938],
        [ 0.9002, -0.5296,  0.0843,  0.2740, -0.3935],
        [ 0.9322, -0.4342,  0.1171,  0.2458, -0.5910],
        [ 0.8779, -0.3307,  0.0430,  0.2211, -0.5863],
        [ 0.6537, -0.2673, -0.0121,  0.3180, -0.3325],
        [ 0.8253, -0.3896,  0.1142,  0.2302, -0.5281],
        [ 0.7816, -0.3399,  0.1225,  0.2413, -0.5844],
        [ 0.9508, -0.5408,  0.1213,  0.2794, -0.4840]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
target: tensor([[0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0.]], device='cuda:0')



  0%|          | 0/10 [02:50<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# Step 4: Initialize the CNN Model
model_2 = CancerCNN_1().float()
print(model_2)

model_2 = model_2.to(device)

# Step 5: Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_2.parameters(), lr=0.0001)

In [ ]:
num_epochs = 25

losses = []


# Train the Model
#try:
#    model_1 = torch.load('Cancer_CNN_1.pt')
#    img = mpimg.imread('loss_1.png')
#    imgplot = plt.imshow(img)
#    plt.show()

#except:
model_2.train()
for epoch in tqdm(range(num_epochs)):  # Number of epochs
    for data, target in tqdm(train_dataloader):
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = model_2(data)
        print(f'output: {output}')
        print(f'target: {target.type_as(output)}\n')
        loss = criterion(output, target.type_as(output)) # Ensuring target is same type as output
        loss.backward()
        optimizer.step()
    losses.append(loss.item())
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')
torch.save(model_2, 'Cancer_CNN_2.pt')
    
fig = plt.figure()
ax = plt.subplot(111)
ax.plot(losses)
plt.title('Loss')
plt.show()
fig.savefig('loss_2.png')